In [ ]:
!pip  install vaderSentiment

In [ ]:
#manipulação dos dados
import pandas as pd
import numpy as np

url_dados = 'https://github.com/jhonatasisraelcl/Data_Science_with_python/blob/main/tweets_Pfizer/vaccination_tweets.csv?raw=true'
df = pd.read_csv(url_dados)

#visualização dos dados
import  missingno
import matplotlib.pyplot as plt
import seaborn as sns

#expressoes regulares
import re
import string

#processamento de linguagem natural
import nltk

nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.stem.porter import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df.head()

In [ ]:
df.shape
#mostra o tamanho do dataframe

In [ ]:
df.info()
#dá todas informações do dataframe
#objeto é um misto de numeros e textos

In [ ]:
df.describe()

# Avaliar dados faltantes

In [ ]:
df.isnull().any()

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
missingno.matrix(df, figsize=(40,10))

In [ ]:
df[df.isnull().any(axis=1)].head()

In [ ]:
linhas, colunas = df.shape
df.shape

In [ ]:
print(linhas)

In [ ]:
percentual_faltantes = (df.isnull().sum()/linhas) * 100

In [ ]:
print(percentual_faltantes)

In [ ]:
df["user_created"] = pd.to_datetime(df["user_created"], format= '%Y-%m-%d %H:%M:%S')
df["date"] = pd.to_datetime(df["date"], format= '%Y-%m-%d %H:%M:%S')


In [ ]:
df.info()

In [ ]:
plt.figure(figsize=(10,12))
sns.barplot(df["user_location"].value_counts().values[0:5], df["user_location"].value_counts().index[0:5]);
plt.title("Top 5 paises", fontsize=12)
plt.xlabel("Numero de tweets", fontsize=12)
plt.ylabel("Paises", fontsize=12)
plt.show

In [ ]:
graf_pizza = df['source'][df['user_location'] == "India"].value_counts().sort_values(ascending=False)
plt.figure(figsize=(10,10))
graf_pizza[0:5].plot(kind="pie", title="Source = India", autopct="%.2f")
plt.show()

In [ ]:
L = ['year', 'month', 'day', 'dayofweek', 'dayofyear', 'weekofyear', 'quarter']


In [ ]:
#df= df.join( pd.concat(( getattr( df['date'], i).rename(i) for i in L), axis=1))
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
df['day'] = pd.DatetimeIndex(df['date']).day
df['dayofweek'] = pd.DatetimeIndex(df['date']).dayofweek
df['dayofyear'] = pd.DatetimeIndex(df['date']).dayofyear
df['weekofyear'] = pd.DatetimeIndex(df['date']).weekofyear
df['quarter'] = pd.DatetimeIndex(df['date']).quarter

#a forma facil nao estava dando certo, ai fiz linha por linha
# https://stackoverflow.com/questions/25146121/extracting-just-month-and-year-separately-from-pandas-datetime-column

In [ ]:
df.head()

In [ ]:
df["text"]

In [ ]:
def limpeza_texto(text):
  text= re.sub('\n', '', text)
  text= re.sub('@[^\s]+', '', text) #apaga o @ e o que vem em seguida
  text= re.sub('\B#\S+', '', text) #apaga o # e o que vem em seguida


  return text

In [ ]:
df['text'] = df['text'].apply(lambda x:limpeza_texto(x))

In [ ]:
df["text"]

In [ ]:
analisa_texto = SentimentIntensityAnalyzer()

In [ ]:
qtd_tweets = (len(df['text']))

In [ ]:
sid = SIA()

In [ ]:
df['sentiments'] = df['text'].apply(lambda x: sid.polarity_scores(' '.join(re.findall(r'\w+', x.lower()))))

In [ ]:
df['sentiments']

In [ ]:
df.head()

In [ ]:
df['positive_sentiment'] = df['sentiments'].apply(lambda x:x ['pos'])

In [ ]:
df['neutral_sentiment'] = df['sentiments'].apply(lambda x:x ['neu'])

In [ ]:
df['negative_sentiment'] = df['sentiments'].apply(lambda x:x ['neg'])

In [ ]:
df['compound'] = df['sentiments'].apply(lambda x:x ['compound'])

In [ ]:
df.head()

In [ ]:
sentimento = []

In [ ]:
for i in df.compound :
  if i >= 0.05:
    sentimento.append("Positive")
  elif i <= (-0.05):
    sentimento.append("Negative")
  else:
    sentimento.append("Neutro")



In [ ]:
print(sentimento)

In [ ]:
df["sentimento"] = pd.Series(np.array(sentimento))

In [ ]:
df.head()

In [ ]:
def limpa_texto(text):
  text = str(text).lower() #deixa em minusculo
  text = re.sub('[%s]' % re.escape(string.punctuation), "", text) #remove pontuações
 
  return text

In [ ]:
df['text'] = df['text'].apply(lambda x: limpa_texto(x))

In [ ]:
def tokenization(text):
  text = re.split('\W+', text)
  return text

  #transforma palavra em um elemento de uma lista

In [ ]:
df_text = pd.DataFrame()

In [ ]:
df_text['text'] = df['text']

In [ ]:
df_text.head(10)

In [ ]:
df_text["tokenization"]  = df_text['text'].apply(lambda x: tokenization(x.lower()))

In [ ]:
df_text.head(10)

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')

In [ ]:
stopwords

In [ ]:
def excluir_stopwords(text):
  text = [ i for i in text if i not in stopwords]
  return text

In [ ]:
df_text["sem_stopwords"] = df_text['tokenization'].apply(lambda x: excluir_stopwords(x))

In [ ]:
df_text

In [ ]:
temp = df.groupby('sentimento').count()['text'].reset_index().sort_values(by="text", ascending=False)
temp.style.background_gradient()